In [1]:
import os
import pandas as pd
from PIL import Image
import timm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from torchvision import models 
from tqdm import tqdm

from albumentations import *
from albumentations.pytorch import ToTensorV2

In [2]:
def get_transforms(need=('train', 'val'), img_size=(512, 384), mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)):
    transformations = {}
    if 'train' in need:
        transformations['train'] = Compose([
            Resize(img_size[0], img_size[1], p=1.0),
            CenterCrop(350,350),
            CLAHE(clip_limit=3.0, tile_grid_size=(8, 8), always_apply=False, p=1.0),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    if 'val' in need:
        transformations['val'] = Compose([
            Resize(img_size[0], img_size[1]),
            CenterCrop(350,350),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    return transformations

In [3]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

In [4]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P40


In [5]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform1 =None, transform2 = None):
        self.img_paths = img_paths
        self.transform1 = transform1
        self.transform2 = transform2
    
    def set_transform(self, transform1, transform2):
        """
        transform 함수를 설정하는 함수입니다.
        """
        self.transform1 = transform1
        self.transform2 = transform2

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform1:
            image_transform1 = self.transform1(image=np.array(image))['image']
        if self.transform2:
            image_transform2 = self.transform2(image = np.array(image))['image']
            
        return image_transform1, image_transform2

    def __len__(self):
        return len(self.img_paths)

In [6]:
class AverageMeter(object):
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [11]:
class MyModel(nn.Module):
    def __init__(self, num_classes: int = 1000):
        super(MyModel, self).__init__()
        self.backbone = models.resnext50_32x4d(pretrained=True)
        self.backbone.fc = nn.Linear(in_features=2048, out_features=18, bias=True)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.backbone(x)
        return x

In [14]:
class MyMultiModel(nn.Module):
    def __init__(self,mask_num_calsses : int = 3, gender_num_calsses : int = 2, age_num_calsses : int = 3):
        super(MyMultiModel, self).__init__()
        self.backbone = models.resnext50_32x4d(pretrained=True)
        self.mask_classifier = nn.Linear(in_features=1000, out_features=mask_num_calsses, bias=True)
        self.gender_classifier = nn.Linear(in_features=1000, out_features=gender_num_calsses, bias=True)
        self.age_classifier = nn.Linear(in_features=1000, out_features=age_num_calsses, bias=True)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.backbone(x)
        z = self.age_classifier(x)
        y = self.gender_classifier(x)
        x = self.mask_classifier(x)

        return x, y, z

In [18]:
# model_path = "/opt/ml/code/backbone/train/2021_04_01/ResNext_2(current best)/v1/004_loss_0.00059.ckpt"
# # model_path = "/opt/ml/pstage1/train/04_03_v1/004_loss_7.7e-05_acc_0.96.ckpt"
# model1 = MyModel()
# model1.load_state_dict(torch.load(model_path))
# model1.cuda()

# model_path = "/opt/ml/model_save/04_04_v1/004_loss_7.5e-05_acc_0.97.ckpt"
# # model_path = "/opt/ml/pstage1/train/04_03_v1/004_loss_7.7e-05_acc_0.96.ckpt"
# model2 = MyModel()
# model2.load_state_dict(torch.load(model_path))
# model2.cuda()

# multi head
model_path = "/opt/ml/model_save/04_08_v4/005_loss_2.1e-05_acc_0.98.ckpt"
model3 = MyMultiModel()
model3.load_state_dict(torch.load(model_path))
model3.cuda()

MyMultiModel(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample):

In [19]:
mean, std = (0.56019358, 0.52410121, 0.501457), (0.23318603, 0.24300033, 0.24567522)

In [20]:
transform = get_transforms(mean=mean, std=std)

In [23]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
dataset = TestDataset(image_paths, transform['val'],transform['train'])

loader = DataLoader(
    dataset,
    shuffle=False
)

# model1.eval()
# model2.eval()
model3.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images1, images2 in tqdm(loader):
    with torch.no_grad():
        images1 = images1.type(torch.FloatTensor).to(device)      
        pred1 = model3(images1) # mask, gender age
        
        images2 = images2.type(torch.FloatTensor).to(device)
        pred2 = model3(images2)
        
        mask_pred = (pred1[0] + pred2[0]) / 2
        gender_pred = (pred1[1] + pred2[1]) / 2
        age_pred = (pred1[2] + pred2[2]) / 2
        
        mask_pred = mask_pred.argmax(dim = -1)
        gender_pred = gender_pred.argmax(dim = -1)
        age_pred = age_pred.argmax(dim = -1)
        
        pred = mask_pred * 6 + gender_pred * 3 + age_pred

        
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

100%|██████████| 12600/12600 [12:55<00:00, 16.24it/s]


test inference is done!
